In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [60]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [61]:
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
    
# get characters that appear in text    
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(f"vocabulary_size: {vocab_size}")

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
vocabulary_size: 81


In [62]:
# tokenizer
# map from str to int and int to str
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [63]:
data = torch.tensor(encode(text), dtype=torch.long) 
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


In [64]:
# train and validation split
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [65]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch("train")
print(f"Inputs: \n{x}")
print(f"Targets: \n{y}")

Inputs: 
tensor([[ 3, 32, 58, 71, 58,  1, 62, 72],
        [73, 61, 62, 72,  1, 62, 72,  9],
        [54, 74, 72, 58,  1, 73, 61, 58],
        [67, 73,  1, 54, 67, 57,  0, 54]], device='cuda:0')
Targets: 
tensor([[32, 58, 71, 58,  1, 62, 72,  1],
        [61, 62, 72,  1, 62, 72,  9,  3],
        [74, 72, 58,  1, 73, 61, 58, 78],
        [73,  1, 54, 67, 57,  0, 54,  1]], device='cuda:0')


In [66]:
# inputs and targets
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, target is {target}")

when input is tensor([80]), target is 0
when input is tensor([80,  0]), target is 1
when input is tensor([80,  0,  1]), target is 1
when input is tensor([80,  0,  1,  1]), target is 28
when input is tensor([80,  0,  1,  1, 28]), target is 39
when input is tensor([80,  0,  1,  1, 28, 39]), target is 42
when input is tensor([80,  0,  1,  1, 28, 39, 42]), target is 39
when input is tensor([80,  0,  1,  1, 28, 39, 42, 39]), target is 44


In [67]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [68]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size)

    
    def forward(self, index, targets=None):
        logits = self.token_embeding_table(index)

        if targets is None:
            loss = None
        else:
            # batch, time, channel
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of idices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1) # (B,T+1)
            
        return index

In [69]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 ImhMUhxImkeUs*mwV.gFRk4pHF909*Mk:1Ws.(68kPhjemh2zqGwT;_:
F94f)6mqkvXKDe.gz;2FQR],8k4gzp jew")_Hdi"*NTHJr3yRTs1BR&cH!﻿Uh;2_dWOv-u4HCqHOhuYh.9;ZH[uGrD?e)zB]zZAVb!8E.DE Io6J86_1e81RbvkvKw
jNtgPX3sN;OB(I4NgLGF!-1"ZcCmIN7l0jB6SSSIjA49;:z3I;-gpi uZALRBuGooaX)m9t!_3xVHajtz;qc3sPGpSmuEI'15U00E0!bvI4yZHVSDJr6olQ3s
[tr
U1ycU7wlQ
U2JW*fW0C
UJH5Yn"vsCVzOhybSlY﻿;lHD0vQ2yL.ggsLW?di _T,jO]vk9!u*P518kuy4r.GUS,)1SDGF5kP2AwDC]LG7?3)7,klQLzB5kX2:jO(5I[8bpSm  -jNu
8
ZA!z7DmEl(Fg&9;G7)DpX5uZAwJpkBiA!,8E!q&kZHMp﻿XSS


In [74]:
# create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"iter: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    # sample batch of data
    xb, yb =  get_batch("train")

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

iter: 0, train loss: 2.6709, val loss: 2.6731
iter: 250, train loss: 2.6171, val loss: 2.7046
iter: 500, train loss: 2.6294, val loss: 2.6926
iter: 750, train loss: 2.6249, val loss: 2.6527
iter: 1000, train loss: 2.6292, val loss: 2.6700
iter: 1250, train loss: 2.6318, val loss: 2.6682
iter: 1500, train loss: 2.5891, val loss: 2.6514
iter: 1750, train loss: 2.6006, val loss: 2.6650
iter: 2000, train loss: 2.6154, val loss: 2.6276
iter: 2250, train loss: 2.5879, val loss: 2.6215
iter: 2500, train loss: 2.5975, val loss: 2.6233
iter: 2750, train loss: 2.6031, val loss: 2.6256
iter: 3000, train loss: 2.5770, val loss: 2.6200
iter: 3250, train loss: 2.5859, val loss: 2.6176
iter: 3500, train loss: 2.5786, val loss: 2.5953
iter: 3750, train loss: 2.5976, val loss: 2.5858
iter: 4000, train loss: 2.5986, val loss: 2.6100
iter: 4250, train loss: 2.5875, val loss: 2.5999
iter: 4500, train loss: 2.5689, val loss: 2.6133
iter: 4750, train loss: 2.5708, val loss: 2.5772
iter: 5000, train loss: 2.

In [75]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



e Wibyhanirere r kPDEE gheayof them tcoorie as E t, oany ainghe
PQSS9zAhamerg houshein wout thes folinoy atin
lllemy in a s kncphowarond Ther ckelvetis;the  doyitt


ED. rss in bal WS?? pXHidins shar, s, ttorereseroshotut bed  wid tror tisthem,"
The't o ilkit g ckiather   baimeabe, PZf previsurk4x:t, beef(﻿er, thas enofl ing
"
g'0_umet theys O]bethe aie ave the ckyerer.wacathedngh aithere t thetofyre a thast  thy f sp;NJis ouly I hellRo ore he im?e twh.
A7D

qRd
BHLT."
WqqL'van,, atondver he th
